In [5]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-tf-deepfm

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded

Step 1/11 : FROM tensorflow/tensorflow:1.10.1-gpu-py3
 ---> 6243acd2b19f
Step 2/11 : WORKDIR /home/tf/
 ---> Using cache
 ---> 814214f6b2ff
Step 3/11 : COPY src/ /home/tf/
 ---> 1eac6057b4c7
Step 4/11 : COPY input/ /opt/ml/input/
 ---> e13b511d9d9e
Step 5/11 : RUN mkdir -p /opt/ml/output/
 ---> Running in cc32a276813e
Removing intermediate container cc32a276813e
 ---> b182309e8241
Step 6/11 : RUN mkdir -p /opt/ml/model/
 ---> Running in f03dbef96b4e
Removing intermediate container f03dbef96b4e
 ---> fd61b79b2fa4
Step 7/11 : RUN apt-get update && apt-get install -y --no-install-recommends 	python3-tk 	&&     	apt-get clean && 	rm -rf /var/lib/apt/lists/*
 ---> Running in cd6966881511
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

